In [1]:
import torch
import torch.nn as nn

In [10]:
in_chan=1
out_chan=1
hidden=8

In [21]:
l1 = nn.Conv2d(in_chan, hidden, 4, 2, 1, bias=False)
l1a = nn.LeakyReLU(0.2, inplace=True)

l2 = nn.Conv2d(hidden, hidden * 2, 4, 2, 1, bias=False)
l2a = nn.BatchNorm2d(hidden * 2)
l2b = nn.BatchNorm2d(hidden * 2)

l3 = nn.Conv2d(hidden * 2, hidden * 4, 4, 2, 1, bias=False)
l3a = nn.BatchNorm2d(hidden * 4)
l3b = nn.LeakyReLU(0.2, inplace=True)

l4 = nn.Conv2d(hidden * 4, 1, 4, 2, 1, bias=False)
l4a = nn.Sigmoid()

x = torch.randn(128, 1, 28, 28)
x = l1(x)
x = l1a(x)
x = l2(x)
x = l2a(x)
x = l2b(x)
x = l3(x)
x = l3a(x)
x = l3b(x)
x = l4(x)
x = l4a(x)
x = x.view(-1, 1).squeeze(1)
print(x.shape)

torch.Size([128])


In [25]:
positive_labels = torch.full((128,), 1.0)
print(positive_labels.shape)

torch.Size([128])


In [26]:
criterion = nn.BCELoss()

In [27]:
criterion(x, positive_labels)

tensor(0.8365, grad_fn=<BinaryCrossEntropyBackward0>)

In [28]:
from gan import Generator

In [29]:
g = Generator(8, 16, 1)

In [33]:
z = torch.randn(128, 8, 1, 1)
x = g(z)
print(x.shape)

torch.Size([128, 1, 28, 28])


In [39]:
x = nn.ConvTranspose2d(8, 8 * 8, 4, 1, 0, bias=False)(z)
x = nn.ConvTranspose2d(hidden * 8, hidden * 4, 4, 2, 1, bias=False)(x)
x = nn.ConvTranspose2d(hidden * 4, hidden * 2, 4, 2, 1, bias=False)(x)
x = nn.ConvTranspose2d(hidden * 2, hidden, 4, 2, 1, bias=False)(x)
x = nn.ConvTranspose2d(hidden, out_chan, kernel_size=1, stride=1, padding=2, bias=False)(x)
print(x.shape)

torch.Size([128, 1, 28, 28])
